In [3]:
# Import libraries

import pandas as pd
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [7]:
path = 'dataset/'
products = 'products-data*.tsv'
reviews = 'reviews-*.tsv'

product_data_files = Path(path).glob(products)
review_files = Path(path).glob(reviews)
df_product_list = []
df_review_list = []